## Get Spotify IDs of MP3s using ACRCloud

### Import stuff

In [4]:
import os
import sys
import hmac
import time
import json
import tqdm
import base64
import hashlib
import urllib.request
import urllib.parse
import datetime
from pydub import AudioSegment

### Functions to perform multipart POST

In [5]:
def post_multipart(url, fields, files):
    content_type, body = encode_multipart_formdata(fields, files)

    req = urllib.request.Request(url, data=body)
    req.add_header('Content-Type', content_type)
    req.add_header('Referer', url)
    resp = urllib.request.urlopen(req)
    ares = resp.read().decode('utf8')
    return ares


def encode_multipart_formdata(fields, files):
    boundary = "*****2016.05.27.acrcloud.rec.copyright." + str(
        time.time()) + "*****"
    body = b''
    CRLF = '\r\n'
    L = []
    for (key, value) in list(fields.items()):
        L.append('--' + boundary)
        L.append('Content-Disposition: form-data; name="%s"' % key)
        L.append('')
        L.append(value)
    body = CRLF.join(L).encode('ascii')
    for (key, value) in list(files.items()):
        L = []
        L.append(CRLF + '--' + boundary)
        L.append('Content-Disposition: form-data; name="%s"; filename="%s"' %
                 (key, key))
        L.append('Content-Type: application/octet-stream')
        L.append(CRLF)
        body = body + CRLF.join(L).encode('ascii') + value
    body = body + (CRLF + '--' + boundary + '--' + CRLF + CRLF).encode('ascii')
    content_type = 'multipart/form-data; boundary=%s' % boundary
    return content_type, body

### Function to identify track and retrieve information

In [6]:
def get_track_info(sample):
    http_method = "POST"
    http_url_file = "/v1/identify"
    data_type = "audio"
    signature_version = "1"
    timestamp = int(
        time.mktime(
            datetime.datetime.utcfromtimestamp(time.time()).timetuple()))
    query_data = sample[:5000000]  # make sure sample is not too big
    sample_bytes = str(len(query_data))

    string_to_sign = http_method + "\n" + http_url_file + "\n" + access_key + "\n" + data_type + "\n" + signature_version + "\n" + str(
        timestamp)
    hmac_res = hmac.new(access_secret.encode('ascii'),
                        string_to_sign.encode('ascii'),
                        digestmod=hashlib.sha1).digest()
    sign = base64.b64encode(hmac_res).decode('ascii')

    fields = {
        'access_key': access_key,
        'sample_bytes': sample_bytes,
        'timestamp': str(timestamp),
        'signature': sign,
        'data_type': data_type,
        "signature_version": signature_version
    }

    res = post_multipart('http://' + host + http_url_file, fields,
                         {"sample": query_data})
    parsed_resp = json.loads(res)
    return parsed_resp

### ACRCloud API credentials

In [22]:
#@markdown Get API credentials from https://www.acrcloud.com/
host = 'fill this in with your details' #@param {type: 'string'}
access_key = 'fill this in with your details' #@param {type: 'string'}
access_secret = 'fill this in with your details' #@param {type: 'string'}

### Scan directory for MP3s

In [8]:
directory = '/path/to/your/mp3s/and/m4as' #@param {'type: 'string'}
ids = {}
mp3s = []
for root, dirs, files in os.walk(directory):
    for file in files:
        if file[-3:] == 'mp3' or file[-3:] == 'm4a':
            mp3s.append(root + '/' + file)

### Compile a list of all the available Spotify IDs

In [26]:
for sound_file in tqdm.tqdm_notebook(mp3s[1971:]):
    if sound_file in ids:
        continue

    try:
        f = open(sound_file, "rb")
        sample = f.read()
        f.close()
        parsed_resp = get_track_info(sample)
        ids[sound_file] = parsed_resp['metadata']['music'][0][
            'external_metadata']['spotify']['track']['id']

    except Exception as e:
        if parsed_resp['status']['code'] == 2004:
            try:
                # re-encode sample as mp3
                audio = AudioSegment.from_file(sound_file,
                                               format=sound_file[-3:])
                audio.export("audio.mp3", format="mp3")
                f = open("audio.mp3", "rb")
                sample = f.read()
                f.close()
                parsed_resp = get_track_info(sample)
                ids[sound_file] = parsed_resp['metadata']['music'][0][
                    'external_metadata']['spotify']['track']['id']
                continue
            except:
                pass

        if 'limit exceeded' in parsed_resp['status']['msg']:
            print(
                f"{parsed_resp['status']['msg']}: Got to {mp3s.index(sound_file)}"
            )
            break
        if parsed_resp['status']['msg'] == 'Success':
            print(f'{e}: Skipping {sound_file}...')
        else:
            print(f"{parsed_resp['status']['msg']}: Skipping {sound_file}...")
        continue

'spotify': Skipping /home/teticio/Music/Music/Compilations/Sound01_ A Big Dada Sampler/08 Communicate.m4a...
'spotify': Skipping /home/teticio/Music/Music/Compilations/Sound01_ A Big Dada Sampler/09 Showtime (Walworth Road Rocker's.m4a...
'spotify': Skipping /home/teticio/Music/Music/Compilations/Sound01_ A Big Dada Sampler/12 The Nonsense (Yeah).m4a...
'spotify': Skipping /home/teticio/Music/Music/Compilations/Strictly Rhythm Est. 1989_ 20 Years Remi/02 Generate Power (Jimpster Mix) [fe.m4a...
'spotify': Skipping /home/teticio/Music/Music/Compilations/Strictly Rhythm Est. 1989_ 20 Years Remi/03 Witch Doktor (Eddie Thoneick Remi.m4a...
'spotify': Skipping /home/teticio/Music/Music/Compilations/Strictly Rhythm Est. 1989_ 20 Years Remi/06 Follow Me (Fred Everything & Oliv.m4a...
'spotify': Skipping /home/teticio/Music/Music/Compilations/Strictly Rhythm Est. 1989_ 20 Years Remi/07 Special (Bingo Players Remix).m4a...
'spotify': Skipping /home/teticio/Music/Music/Compilations/Strictly Rhyt

'spotify': Skipping /home/teticio/Music/Music/Compilations/Worldwide [Disc 1]/1-05 Easin' In.m4a...
No result: Skipping /home/teticio/Music/Music/Compilations/Worldwide [Disc 1]/1-07 Lonely Can Be Sweet.m4a...
'spotify': Skipping /home/teticio/Music/Music/Compilations/Worldwide [Disc 1]/1-10 The Last Beat (House Mix).m4a...
No result: Skipping /home/teticio/Music/Music/Compilations/Worldwide [Disc 1]/1-11 Thinking Louder.m4a...
No result: Skipping /home/teticio/Music/Music/Compilations/Worldwide [Disc 1]/1-12 Mind Of A Killer.m4a...
No result: Skipping /home/teticio/Music/Music/Compilations/Worldwide [Disc 1]/1-13 Roun Soun.m4a...
No result: Skipping /home/teticio/Music/Music/Compilations/Worldwide [Disc 2]/2-01 The Mystery Of Man.m4a...
'spotify': Skipping /home/teticio/Music/Music/Compilations/Worldwide [Disc 2]/2-02 This World.m4a...
No result: Skipping /home/teticio/Music/Music/Compilations/Worldwide [Disc 2]/2-04 Black Mary.m4a...
No result: Skipping /home/teticio/Music/Music/Comp

'spotify': Skipping /home/teticio/Music/Music/David Matthews/Dune-(CTI-7-5005) LP/B2-Silent Running-(Peter Schic.mp3...
No result: Skipping /home/teticio/Music/Music/David Matthews/Dune-(CTI-7-5005) LP/B3-Princess Leis Theme-(John W.mp3...
'spotify': Skipping /home/teticio/Music/Music/David Matthews/Dune-(CTI-7-5005) LP/B4-Star Wars-(John Williams).mp3...
No result: Skipping /home/teticio/Music/Music/De La Soul/3 Feet High and Rising/09 Eye Know.mp3...
'spotify': Skipping /home/teticio/Music/Music/Deadbeats/Paul's First Compilation/03 Got it Going.mp3...
'spotify': Skipping /home/teticio/Music/Music/Dee Dee Bridgewater/Afro Blue/07 Peaple Make The World Go Around.mp3...
'spotify': Skipping /home/teticio/Music/Music/Deep Blue/Helicopter '97/Thursday.mp3...
No result: Skipping /home/teticio/Music/Music/Deep Sang/Bump N Hustle/01 Rhythm Makers ... Zone.mp3...
No result: Skipping /home/teticio/Music/Music/Deep Sang/Bump N Hustle/02 Cerrone ... Hooked On You.mp3...
No result: Skipping /home

'spotify': Skipping /home/teticio/Music/Music/Duran Y Garcia/Nu 54 Entertainmentbar Vol. 1 Mixed By S/Old Fashioned Thief.mp3...
'spotify': Skipping /home/teticio/Music/Music/Dusko Gojkovic/Trumpets & Rhythm Unit/01 Quo Vadis Samba.mp3...
No result: Skipping /home/teticio/Music/Music/Dusko Gojkovic/Trumpets & Rhythm Unit/04 Rufus.mp3...
No result: Skipping /home/teticio/Music/Music/Dusko Gojkovic/Trumpets & Rhythm Unit/06 The Ambassador.mp3...
No result: Skipping /home/teticio/Music/Music/Dusko Gojkovic/Trumpets & Rhythm Unit/09 Majority Of One.mp3...
'spotify': Skipping /home/teticio/Music/Music/Dwele/Gilles Peterson Presents_ The BBC Sessio/2-07 Red Clay.mp3...
'spotify': Skipping /home/teticio/Music/Music/Dynamic Syncopation/Dynamism/01 Intro.mp3...
'spotify': Skipping /home/teticio/Music/Music/Dynamic Syncopation/Dynamism/13 Swings & Roundabouts.mp3...
No result: Skipping /home/teticio/Music/Music/Dynamite Brass/Unknown Album/Fura de Fira.mp3...
'spotify': Skipping /home/teticio/Mu

'spotify': Skipping /home/teticio/Music/Music/Faithless/Reverence/Insomnia.mp3...
'spotify': Skipping /home/teticio/Music/Music/Fantastic Plastic Machine/Morricone Rmx/Belinda May (Edit).mp3...
'spotify': Skipping /home/teticio/Music/Music/Fantastic Plastic Machine/Space Baby Blast Off/10 Bachelor Pad.mp3...
No result: Skipping /home/teticio/Music/Music/Fantastic Plastic Machine/The Fantastic Plastic Machine/Bon Voyage.mp3...
'spotify': Skipping /home/teticio/Music/Music/Fantastic Plastic Machine/The Fantastic Plastic Machine/Please, Stop!.mp3...
'spotify': Skipping /home/teticio/Music/Music/Fantastic Plastic Machine/Too/Spectacular.mp3...
'spotify': Skipping /home/teticio/Music/Music/Fantastic Plastic Machine/ソトシゴト m-flo turns it out! (disc 1)/Beautiful Days (TAKU'S GARAGE mix).mp3...
'spotify': Skipping /home/teticio/Music/Music/Fat Freddy's Drop/Gilles Peterson Presents - The BBC Sessi/1-13 This Room.mp3...
No result: Skipping /home/teticio/Music/Music/Fat Freddys Drop/Record-Play p

...

No result: Skipping /home/teticio/Music/Music/Unknown Artist/Unknown Album/20130203 180844.m4a...
No result: Skipping /home/teticio/Music/Music/Unknown Artist/Unknown Album/20130218 132543.m4a...
No result: Skipping /home/teticio/Music/Music/Unknown Artist/Unknown Album/20130218 132824.m4a...
No result: Skipping /home/teticio/Music/Music/Unknown Artist/Unknown Album/20130218 132938.m4a...
No result: Skipping /home/teticio/Music/Music/Unknown Artist/Unknown Album/20130808 230742.m4a...
No result: Skipping /home/teticio/Music/Music/Unknown Artist/Unknown Album/20130808 231942.m4a...
No result: Skipping /home/teticio/Music/Music/Unknown Artist/Unknown Album/20130808 233536.m4a...
No result: Skipping /home/teticio/Music/Music/Unknown Artist/Unknown Album/20130808 235420.m4a...
No result: Skipping /home/teticio/Music/Music/Unknown Artist/Unknown Album/20131219 221347.m4a...
No result: Skipping /home/teticio/Music/Music/Unknown Artist/Unknown Album/20150617 182832.m4a...
No result: Skipping 

No result: Skipping /home/teticio/Music/Music/Unknown Artist/Unknown Album/51 Atlas Shrugged - 51.mp3...
No result: Skipping /home/teticio/Music/Music/Unknown Artist/Unknown Album/52 Atlas Shrugged - 52 1.mp3...
No result: Skipping /home/teticio/Music/Music/Unknown Artist/Unknown Album/52 Atlas Shrugged - 52.mp3...
No result: Skipping /home/teticio/Music/Music/Unknown Artist/Unknown Album/53 Atlas Shrugged - 53 1.mp3...
No result: Skipping /home/teticio/Music/Music/Unknown Artist/Unknown Album/53 Atlas Shrugged - 53.mp3...
No result: Skipping /home/teticio/Music/Music/Unknown Artist/Unknown Album/54 Atlas Shrugged - 54 1.mp3...
No result: Skipping /home/teticio/Music/Music/Unknown Artist/Unknown Album/54 Atlas Shrugged - 54.mp3...
No result: Skipping /home/teticio/Music/Music/Unknown Artist/Unknown Album/55 Atlas Shrugged - 55 1.mp3...
No result: Skipping /home/teticio/Music/Music/Unknown Artist/Unknown Album/55 Atlas Shrugged - 55.mp3...
No result: Skipping /home/teticio/Music/Music/U

No result: Skipping /home/teticio/Music/Music/Unknown Artist/Unknown Album/89 Atlas Shrugged - 89 1.mp3...
No result: Skipping /home/teticio/Music/Music/Unknown Artist/Unknown Album/89 Atlas Shrugged - 89.mp3...
No result: Skipping /home/teticio/Music/Music/Unknown Artist/Unknown Album/9. Presidente1_con_aplausos.mp3...
No result: Skipping /home/teticio/Music/Music/Unknown Artist/Unknown Album/90 Atlas Shrugged - 90 1.mp3...
No result: Skipping /home/teticio/Music/Music/Unknown Artist/Unknown Album/90 Atlas Shrugged - 90.mp3...
No result: Skipping /home/teticio/Music/Music/Unknown Artist/Unknown Album/91 Atlas Shrugged - 91 1.mp3...
No result: Skipping /home/teticio/Music/Music/Unknown Artist/Unknown Album/91 Atlas Shrugged - 91.mp3...
No result: Skipping /home/teticio/Music/Music/Unknown Artist/Unknown Album/92 Atlas Shrugged - 92.mp3...
No result: Skipping /home/teticio/Music/Music/Unknown Artist/Unknown Album/93 Atlas Shrugged - 93.mp3...
No result: Skipping /home/teticio/Music/Musi

No result: Skipping /home/teticio/Music/Music/Unknown Artist/Unknown Album/You-Are-Not-Your-Pain-Audio-Excerpt- 1.mp3...
No result: Skipping /home/teticio/Music/Music/Unknown Artist/Unknown Album/You-Are-Not-Your-Pain-Audio-Excerpt- 2.mp3...
No result: Skipping /home/teticio/Music/Music/Unknown Artist/Unknown Album/You-Are-Not-Your-Pain-Audio-Excerpt- 3.mp3...
No result: Skipping /home/teticio/Music/Music/Unknown Artist/Unknown Album/You-Are-Not-Your-Pain-Audio-Excerpt- 4.mp3...
No result: Skipping /home/teticio/Music/Music/Unknown Artist/Unknown Album/You-Are-Not-Your-Pain-Audio-Excerpt- 5.mp3...
No result: Skipping /home/teticio/Music/Music/Unknown Artist/Unknown Album/You-Are-Not-Your-Pain-Audio-Excerpt- 6.mp3...
No result: Skipping /home/teticio/Music/Music/Unknown Artist/Unknown Album/You-Are-Not-Your-Pain-Audio-Excerpt- 7.mp3...
No result: Skipping /home/teticio/Music/Music/Unknown Artist/Unknown Album/You-Are-Not-Your-Pain-Audio-Excerpt-.mp3...
No result: Skipping /home/teticio/

'spotify': Skipping /home/teticio/Music/Music/阿￝￰子/MISS-A-MIXTURE/07 SEND ONE YOUR LOVE_ New Perspecti.mp3...
'spotify': Skipping /home/teticio/Music/Music/阿￝￰子/MISS-A-MIXTURE/08 STOP IN THE NAME OF LOVE_ N. Y. M.mp3...
'spotify': Skipping /home/teticio/Music/Music/阿￝￰子/MISS-A-MIXTURE/09 CAMALEO [CHAMELEON]_ Dub Mix.mp3...
'spotify': Skipping /home/teticio/Music/Music/阿￝￰子/MISS-A-MIXTURE/10 Ooo BABY BABY_ N. Y. Mixture.mp3...
'spotify': Skipping /home/teticio/Music/Music/阿￝￰子/MISS-A-MIXTURE/11 MORE AND MORE_ Cold Fusion Mix.mp3...
'spotify': Skipping /home/teticio/Music/Music/阿￝￰子/MISS-A-MIXTURE/12 SKINDO-LE-LE_ GRP Remix.mp3...
'spotify': Skipping /home/teticio/Music/Music/阿￝￰子/MISS-A-MIXTURE/13 DUKE'S PLACE_ Electric Bop Mix.mp3...


## Add all the Spotify tracks to a playlist

### Import more stuff

In [27]:
import spotipy
import spotipy.util as util

### Spotify API credentials

In [28]:
scope = 'playlist-modify-public'
#@markdown Get API credentials from https://developer.spotify.com/dashboard/login
client_id = 'fill this in with your details' #@param {type: 'string'}
client_secret = 'fill this in with your details' #@param {type: 'string'}
#@markdown The callback URI has to allow you to get hold of the token (e.g. https://github.com/)
redirect_uri = 'fill this in with your details' #@param {type: 'string'}
username = 'fill this in with your details' #@param {type: 'string'}
playlist_name = 'fill this in with your details' #@param {type: 'string'}

In [31]:
# Spotipy function is currently broken

def user_playlist_create(sp,
                         username,
                         playlist_name,
                         description='',
                         public=True):
    data = {
        'name': playlist_name,
        'public': public,
        'description': description
    }
    return sp._post("users/%s/playlists" % (username, ), payload=data)['id']

In [35]:
token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)
sp = spotipy.Spotify(token)
playlists = sp.user_playlists(username)
playlist_ids = [playlist['id'] for playlist in playlists['items'] if playlist['name'] == playlist_name]
if len(playlist_ids) == 0:
    user_playlist_create(sp, username, playlist_name)
else:
    playlist_ids = playlist_id

In [30]:
tracks = []
replace = True
for id in ids:
    tracks.append(ids[id])
    if len(tracks) == 100:
        if replace:
            sp.user_playlist_replace_tracks(username, playlist_id, tracks)
            replace = False
        else:
            sp.user_playlist_add_tracks(username, playlist_id, tracks)
        tracks = []